In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Load numpy array

In [3]:
text_npy = np.load('./npy/text.npy')
poster_npy = np.load('./npy/poster.npy')
IGimg_npy = np.load('./npy/IGimg.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre.npy')

print('Text:', text_npy.shape)
print('Poster:', poster_npy.shape)
print('IGimg:', IGimg_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

Text: (166, 300)
Poster: (166, 256)
IGimg: (166, 256)
Movie genre: (166, 20)
User following: (2020, 166)
User genre: (2020, 20)


### Normalize usr_genre

In [4]:
usr_genre_norm = np.zeros(usr_genre.shape)
for i in range(len(usr_genre)):
    usr_genre_norm[i] = usr_genre[i]/np.max(usr_genre[i])
print(usr_genre_norm.shape)

(2020, 20)


In [5]:
print('Before:', usr_genre)
print('After:', usr_genre_norm)

Before: [[2 1 0 ... 1 0 0]
 [0 0 0 ... 1 0 1]
 [3 7 4 ... 0 0 0]
 ...
 [5 3 0 ... 1 1 0]
 [2 2 0 ... 0 1 0]
 [2 1 0 ... 1 1 0]]
After: [[0.22222222 0.11111111 0.         ... 0.11111111 0.         0.        ]
 [0.         0.         0.         ... 0.125      0.         0.125     ]
 [0.33333333 0.77777778 0.44444444 ... 0.         0.         0.        ]
 ...
 [0.26315789 0.15789474 0.         ... 0.05263158 0.05263158 0.        ]
 [0.28571429 0.28571429 0.         ... 0.         0.14285714 0.        ]
 [0.22222222 0.11111111 0.         ... 0.11111111 0.11111111 0.        ]]


# Training & testing split

### Setup 

In [6]:
usr_test_amount = 200
movie_test_amount = 30

print(usr_test_amount, movie_test_amount)

200 30


In [7]:
usr_idx = [i for i in range(len(usr_following))]
print(len(usr_idx))

test_idx = random.sample(usr_idx, usr_test_amount)
print(len(test_idx))

2020
200


In [8]:
#Training
train_t = [0] * (len(usr_following))
train_f = [0] * (len(usr_following))
# Testing
test_t = [0] * usr_test_amount
test_f = [0] * usr_test_amount
test_pos = -1

for i in range(len(usr_following)):
    
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(166):
            if usr_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        
        for j in range(166):
            
            if usr_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        
        # random choose 2 true and 8 false for test 
        t_for_test = random.sample(temp_t, 2)
        f_for_test  = random.sample(temp_f, 8)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [29]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_f))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 2020
The length of train_f: 2020
The length of test_t: 200
The length of test_f: 200


# Recommendation model

In [10]:
usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies
laten_dim = 32 # latent dims
ft_dim = 256 # feature dims
embedding_dims = 150

In [11]:
user = tf.placeholder(tf.int32,[None])
i = tf.placeholder(tf.int32, [None])
j = tf.placeholder(tf.int32, [None])

#多少個auxliary 
xf = tf.placeholder(tf.float32, [None, ft_dim])
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))

image_i = tf.placeholder(tf.float32, [1, ft_dim])
image_j = tf.placeholder(tf.float32, [1, ft_dim])

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [usr_nb, laten_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [movie_nb, laten_dim],
                                  initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [movie_nb, laten_dim],
                               initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [usr_nb, laten_dim],
                         initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [laten_dim,laten_dim], 
                         initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [laten_dim, laten_dim],
                         initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [movie_nb, laten_dim, laten_dim],
                         initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [laten_dim, ft_dim],
                         initializer=tf.contrib.layers.xavier_initializer())
    
    
    aux_new = tf.get_variable("aux_new", [ft_dim, laten_dim], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    Beta = tf.get_variable("beta", [usr_nb, ft_dim],
                           initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))
    
#     embedding = tf.get_variable("embedding", [embedding_dims, ft_dim],
#                                 initializer=tf.contrib.layers.xavier_initializer())
#     Beta = tf.get_variable("beta", [usr_nb, embedding_dims],
#                            initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i)
vj = tf.nn.embedding_lookup(item_latent, j)

#取消 Weight 共用
w1 = tf.nn.embedding_lookup(W1, user)
wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user))
wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user))
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user))
wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user))

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [34]:
a_list=[]

for q in range(3): #取l_id個 YouTuber 的 類別
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    a_list.append((tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q])
        
                                          
a_list_soft=tf.nn.softmax(a_list)
#print(sess.run(a_list_soft))
aux_np = np.zeros(laten_dim)

#改成while
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)

aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

#矩陣中對應函數各自相乘
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,image_i, transpose_b=True)
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,image_j, transpose_b=True)

xuij = xui- xuj

l2_norm = tf.add_n([
            0.001 * tf.reduce_sum(tf.multiply(u, u)),
            0.001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.001 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.001 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.001 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.1 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

In [35]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0 = time.time()

train_pair_t = [] #positive feedback
train_pair_f = [] #negative feedback
train_yes_id = [] 
for q in range(10):
    print('Iteraction:',q)
    train_auc = 0
    total_loss = 0
    xuij_auc = 0
    length = 0
    
    for z in range(usr_nb):
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        r_3=np.zeros(3) 
        
        #這裡不知道怎麼讓3變成變動型的長度
        sample=random.sample(train_t[z],3) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        for k in range(len(sample)):
            yes.append(IGimg_npy[sample[k]])
            yesr.append(movie_genre[sample[k]]*usr_genre_norm[z])
#             print('movie_genre:', movie_genre[sample[k]])
#             print('usr_genre_norm:',usr_genre_norm[z])
            
        for k in range(3):
            r_3[k]=max(yesr[k])
        yes=np.array(yes)
        
        not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        train_t_sample = random.sample(train_t[z],2)
        #print('number of positive feedback', len(train_t[z]))
        for ta in train_t_sample:
            #ta=random.choice(train_t[z]) #ta is true positve photo
            train_pair_t.append(ta)
            image_1=np.expand_dims(IGimg_npy[ta],0)
            
            train_f_sample = random.sample(train_f[z],10)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                train_pair_f.append(b)
                image_2=np.expand_dims(IGimg_npy[b],0)
                #print('Image_2',image_2.shape)
            
                #use_test[z].append(b)
                r3,_auc, _loss,_=sess.run([a_list_soft,auc,loss,train_op], 
                                          feed_dict={user: [z], i: [ta], j: [b],
                                                     xf: yes , l_id:sample,r:r_3,
                                                     image_i:image_1,image_j:image_2})
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                #print('sub r3:',r3)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')

Iteraction: 0


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype int32 and shape [1]
	 [[node Placeholder_5 (defined at <ipython-input-11-4ff585d6bca2>:8) ]]
	 [[node Mean_1 (defined at <ipython-input-34-0258b46654f8>:45) ]]

Caused by op 'Placeholder_5', defined at:
  File "/home/tonylab/miniconda3/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tonylab/miniconda3/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/tonylab/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/tonylab/miniconda3

In [ ]:
image_2.shape

In [ ]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

In [ ]:
def training(save_name): 
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()

    train_yes_id=[] 
    for q in range(7):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(usr_nb):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]

            """
            only choose positive 
            """
            #sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            """
            choose all YouTuber 
            """
            sample= [i for i in range(movie_nb)] #選全部的Positive

            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))

            #user degree of category favor 
            r_3=np.zeros(len(sample)) 

            for b in range(len(sample)):
                yes.append(IGimg_npy[sample[b]])
                yesr.append(movie_genre[sample[b]] * usr_genre_norm[z])

            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)

            yes=np.array(yes)

            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))

            #train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                pos = sample.index(ta)

                image_1=np.expand_dims(IGimg_npy[ta],0) #(1,2048)
                train_f_sample = random.sample(train_f[z],10)

                for b in train_f_sample:
                    image_2=np.expand_dims(IGimg_npy[b],0) #(1,2048)
                    _last_be_relu,_norm_par,_a_list,r3,_auc, _loss,_ = sess.run(
                        [last_be_relu,norm_par,a_list_smooth,a_list_soft,auc,loss,train_op], 
                        feed_dict = {user: [z],i: [ta], j: [b], xf: yes , 
                                     l_id:sample, l_id_len:[len(sample)],
                                     positive_id:train_t[z], positive_len:[len(train_t[z])],
                                     r:r_3, image_i:image_1, image_j:image_2})

                    #print(_a_list)
                    #print(r3)
                    train_auc += _auc
                    total_loss += _loss
                    length += 1
        #print('a_list:',_a_list)
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, Au, Ay, Aa, Av,B = sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])

    np.savez('./output/'+save_name+'.npz',
             U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)

    return U, Y, A, E, Au, Ay, Aa, Av,B

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]

no_feature = 'only_Image'

testcount = 0
finish_list = []
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished Dims',finish_list)
                    finish_list.append(embedding_dims)
                    print('Now Dims:',embedding_dims)
                    
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,ft_dim))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,ft_dim))
                    image_j = tf.placeholder(tf.float32, shape=(1,ft_dim))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [usr_nb, laten_dim],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [movie_nb, laten_dim],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [movie_nb, laten_dim],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [usr_nb, movie_nb, laten_dim],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [usr_nb, movie_nb, laten_dim],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [usr_nb, movie_nb, laten_dim],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [usr_nb, movie_nb, embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,laten_dim], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,ft_dim],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [usr_nb,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(laten_dim),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training('Img0.5_'+no_feature+'_E200')
                    print('Finish dims:,',embedding_dims)

# Get latent factor and Each weight

In [ ]:
U, M, A, A1, Au, Am, Aa, Av,B = sess.run([user_latent, item_latent, aux_item, W1, Wu, Wm, Wa, Wv,Beta])

In [ ]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

# Testing Part

In [ ]:
result=np.zeros((200, 166))
RS=np.zeros((200, 166))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(200):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(movie_genre[sample[a]] * usr_genre_norm[test_idx[s]]) #sample a 的category vec *user_category vec
    
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(IGimg_npy[sample[a]],0).T))))*r
    mul=np.zeros((1,32))
    #print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(166):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], IGimg_npy[k].T)
print(RS[s])

In [ ]:
#取出test的資料
testRS = np.zeros((200,10)) #shape 200*10

#test_t 是true的
#test_f 是false的
        
for z in range(200):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [ ]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [ ]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),2)
    count_0_all.append(top_0)
    #print(top_0)

acc_0 = 0
total = len(count_0_all)*len(count_0_all[0])
#print(total) #(200*2)
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        if count_0_all[i][j] < 2: #代表是0或1 (也就是target)
            acc_0 += 1
#print(acc_0)
avg_acc = acc_0/total
print('avg_accuarcy for count_0:',avg_acc)

# Testing for dynamic length for loop

In [ ]:
length = tf.placeholder(tf.int32)
i = tf.constant(0)

while_condition = lambda i: tf.less(i, length)
def body(i):
    # do something here which you want to do in your loop
    # increment i
    return [tf.add(i, 1)]

# do the loop:
output = tf.while_loop(while_condition, body, [i])


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(output,feed_dict={length:3})